In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama

gemma_llm = ChatGroq(model="gemma2-9b-it")
ollama_llm = ChatOllama(model="llama3.2:1b")

In [8]:
from langchain_core.prompts import PromptTemplate

def few_shot_sentiment_classification(input_text):
    few_shot_prompt = PromptTemplate(
        input_variables=["input_text"],
        template="""
        Classify the sentiment as Positive, Negative, or Neutral.
        
        Examples:
        Text: I love this product! It's amazing.
        Sentiment: Positive
        
        Text: This movie was terrible. I hated it.
        Sentiment: Negative
        
        Text: The weather today is okay.
        Sentiment: Neutral
        
        Now, classify the following:
        Text: {input_text}
        Sentiment:
        """
    )
    
    chain = few_shot_prompt | ollama_llm
    result = chain.invoke(input_text).content
    result = result.strip()
    if ':' in result:
        result = result.split(':')[1].strip()
    
    return result

In [9]:
test_text = "I can't believe how great this new restaurant is!"
result = few_shot_sentiment_classification(test_text)
print(f"Input: {test_text}")
print(f"Predicted Sentiment: {result}")

Input: I can't believe how great this new restaurant is!
Predicted Sentiment: To classify the sentiment of the text "I can't believe how great this new restaurant is!", I would say that it is Positive.

The word "great" indicates a strong positive feeling, and the phrase is used to express amazement or enthusiasm. The use of all capital letters for "can't" also adds emphasis to the negative aspect, indicating disappointment or surprise rather than actual inability.


#Advanced Few-Shot Prompting

In [10]:
def multi_task_few_shot(input_text, task):
    few_shot_prompt = PromptTemplate(
        input_variables=["input_text", "task"],
        template="""
        Perform the specified task on the given text.
        
        Examples:
        Text: I love this product! It's amazing.
        Task: sentiment
        Result: Positive
        
        Text: Bonjour, comment allez-vous?
        Task: language
        Result: French
        
        Now, perform the following task:
        Text: {input_text}
        Task: {task}
        Result:
        """
    )
    
    chain = few_shot_prompt | ollama_llm
    return chain.invoke({"input_text": input_text, "task": task}).content

In [11]:
print(multi_task_few_shot("I can't believe how great this is!", "sentiment"))
print(multi_task_few_shot("Guten Tag, wie geht es Ihnen?", "language"))

To determine the sentiment of the given text "I can't believe how great this is!", we need to analyze its emotional tone. Here's the breakdown:

1. The word "can't" typically indicates a negative emotion, so it adds to the overall negative sentiment.
2. The word "believe" is often associated with optimism or confidence, which contradicts the negative sentiment caused by the first two words.

Given these factors, I would classify the sentiment of this text as Negative.
I can perform the tasks on the given text.

Text: Guten Tag, wie geht es Ihnen?

Task: language
Result: German


#In-Content Learning

In [12]:
def in_context_learning(task_description, examples, input_text):
    example_text = "".join([f"Input: {e['input']}\nOutput: {e['output']}\n\n" for e in examples])
    
    in_context_prompt = PromptTemplate(
        input_variables=["task_description", "examples", "input_text"],
        template="""
        Task: {task_description}
        
        Examples:
        {examples}
        
        Now, perform the task on the following input:
        Input: {input_text}
        Output:
        """
    )
    
    chain = in_context_prompt | ollama_llm
    return chain.invoke({"task_description": task_description, "examples": example_text, "input_text": input_text}).content

In [13]:
task_desc = "Convert the given text to pig latin."
examples = [
    {"input": "hello", "output": "ellohay"},
    {"input": "apple", "output": "appleay"}
]
test_input = "python"

result = in_context_learning(task_desc, examples, test_input)
print(f"Input: {test_input}")
print(f"Output: {result}")

Input: python
Output: To convert the given text into Pig Latin, we will follow these steps:

1. Take the first letter of each word as it is.
2. Move all other letters to the end of the word.

The rules for Pig Latin are as follows:
- If a word begins with a vowel (a, e, i, o, u), you simply move all the consonants to the end of the word and add "way".
- If a word begins with a consonant but ends with a vowel, you move all the consonants to the end of the word and add "ay".

Now, let's apply these rules on the input "python":

1. P becomes y (first letter)
2. N is not the first letter, so we skip it
3. Y is also not the first letter, but it does move to the end: ythonpy

So the Pig Latin translation of "python" would be "ythonypay".


#Practice and Evaluation

In [14]:
def evaluate_model(model_func, test_cases):
    '''
    Evaluate the model on a set of test cases.

    Args:
    model_func: The function that makes predictions.
    test_cases: A list of dictionaries, where each dictionary contains an "input" text and a "label" for the input.

    Returns:
    The accuracy of the model on the test cases. 
    '''
    correct = 0
    total = len(test_cases)
    
    for case in test_cases:
        input_text = case['input']
        true_label = case['label']
        prediction = model_func(input_text).strip()
        
        is_correct = prediction.lower() == true_label.lower()
        correct += int(is_correct)
        
        print(f"Input: {input_text}")
        print(f"Predicted: {prediction}")
        print(f"Actual: {true_label}")
        print(f"Correct: {is_correct}\n")
    
    accuracy = correct / total
    return accuracy

In [15]:
test_cases = [
    {"input": "This product exceeded my expectations!", "label": "Positive"},
    {"input": "I'm utterly disappointed with the service.", "label": "Negative"},
    {"input": "The temperature today is 72 degrees.", "label": "Neutral"}
]

accuracy = evaluate_model(few_shot_sentiment_classification, test_cases)
print(f"Model Accuracy: {accuracy:.2f}")

Input: This product exceeded my expectations!
Predicted: Sentiment
Actual: Positive
Correct: False

Input: I'm utterly disappointed with the service.
Predicted: 1. "I love this product! It's amazing." - Positive

2. "This movie was terrible. I hated it." - Negative

3. "The weather today is okay." - Neutral

4. "I'm utterly disappointed with the service." - Negative

Since all four texts express a negative sentiment, I would classify their sentiment as Negative.
Actual: Negative
Correct: False

Input: The temperature today is 72 degrees.
Predicted: - Text
Actual: Neutral
Correct: False

Model Accuracy: 0.00
